In [1]:
# nuclio: ignore
import nuclio

In [ ]:
%nuclio config kind="serving"
%nuclio env MODEL_CLASS="RNN_Model_Serving"
%nuclio config spec.build.baseImage = "mlrun/mlrun"
%nuclio cmd python -m pip install tensorflow
%nuclio cmd python -m pip install keras

In [ ]:
!pip install storey==0.4.2

In [4]:
import os
import mlrun
import numpy as np
import keras
import json
import requests
import warnings 
warnings.filterwarnings('ignore')

In [5]:
class RNN_Model_Serving(mlrun.serving.V2ModelServer):
    def load(self):
        """load and initialize the model and/or other elements"""
        model_file,extra_data = self.get_model(suffix = ".h5")
        self.model = keras.models.load_model(model_file)

    def predict(self, body: dict) -> list:
        """Generate model predictions from sample"""
        feats = np.asarray(body['inputs'])
        result = self.model.predict(feats)
        result = json.dumps(result.tolist())
        return result

In [6]:
# nuclio: end-code

In [7]:
# test localy

In [8]:
import mlrun

fn = mlrun.new_function('RNN_serving_function', kind='serving')

# set the topology/router and add models
fn.add_model("RNN_stocks", class_name="RNN_Model_Serving", model_path="/v3io/users/dani/test/demos/stock-analysis/models/")

# create and use the graph simulator
server = fn.to_mock_server()

> 2021-04-29 08:54:10,487 [info] model RNN_stocks was loaded
> 2021-04-29 08:54:10,488 [info] Loaded ['RNN_stocks']


In [12]:
# test remotely

In [13]:
from mlrun import new_model_server, mount_v3io
import requests
import os

In [14]:
from mlrun import code_to_function
fn = code_to_function('RNN_serving_function', kind='serving')
fn.add_model("RNN_stocks", class_name="RNN_Model_Serving", model_path="/v3io/users/dani/test/demos/stock-analysis/models/")
fn.apply(mount_v3io())

In [15]:
addr = fn.deploy()

> 2021-04-29 08:54:16,271 [info] Starting remote function deploy
2021-04-29 08:54:16  (info) Deploying function
2021-04-29 08:54:16  (info) Building
2021-04-29 08:54:16  (info) Staging files and preparing base images
2021-04-29 08:54:16  (info) Building processor image
2021-04-29 08:54:20  (info) Build complete
2021-04-29 08:54:32  (info) Function deploy complete
> 2021-04-29 08:54:33,385 [info] function deployed, address=default-tenant.app.dev8.lab.iguazeng.com:32458
